In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [7]:
def load_data(movie_path, rating_path):

    movie = pd.read_csv(movie_path)
    rating = pd.read_csv(rating_path)
    movie = movie.drop_duplicates()
    rating = rating.drop_duplicates()
    return movie, rating

def data_prep(movie_data, rating_data):

    data = rating_data.merge(movie_data, on = 'movieId', how = 'inner')
    group_df = data.groupby(['movieId']).count()
    idx = group_df[group_df.userId > 1000].index.to_list()
    data = data[data['movieId'].isin(idx)]
    return data


def recommendation_engine(data,  user_id, rating, movie):
    print('Start Recommendation')

    data_pivot_table = data.pivot_table(index='userId', columns= 'title', values='rating')
    print('Pivot complete..')

    try:
    # random_user = int(pd.Series(data_pivot_table.index).sample(1, random_state=555).values)
        user_df = data_pivot_table[data_pivot_table.index == user_id]
        user_watched_movie = user_df.columns[[user_df.notna().any()]].to_list()

        movies_watched_df = data_pivot_table[user_watched_movie]
        users_movie_count = movies_watched_df.T.notnull().sum().reset_index()
        users_movie_count.columns = ["userId", "movie_count"]
        mean_value = np.percentile(users_movie_count['movie_count'],95)
        movie_ids = users_movie_count[users_movie_count['movie_count'] > mean_value]['userId'].values.tolist()
        new_df = movies_watched_df[movies_watched_df.index.isin(movie_ids)]
        print('Starting corr')
        corr_df = new_df.T.corr().unstack().sort_values().drop_duplicates()
        print('Corr complete..')

        corr_df = pd.DataFrame(corr_df, columns=["corr"])
        corr_df.index.names = ['random_user', 'other_users']
        corr_df.reset_index(inplace=True)
        
        mean_value = np.percentile((corr_df['corr']),75)
        top_users = corr_df[(corr_df['random_user']==user_id)&(corr_df['corr']>=mean_value)]
        top_users = top_users.loc[:, ['other_users','corr']].rename(columns={"other_users": "userId"})
        top_users.reset_index(inplace=True, drop=True)
        try:
            top_users_ratings = top_users.merge(rating, how='inner', on='userId')
            top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
            recommendation_df = top_users_ratings.groupby('movieId')[['weighted_rating']].mean().reset_index()
            mean_value = np.percentile(recommendation_df['weighted_rating'],95)
            print(recommendation_df.shape, mean_value)
            final_recommendation_df=recommendation_df[recommendation_df['weighted_rating']>mean_value].sort_values(by='weighted_rating', ascending=False)
            recommendation = final_recommendation_df.merge(movie[["movieId", "title"]])[:5]
            titles = recommendation['title'].tolist()
        # movies_watched_df = data_pivot_table[user_watched_movie]
        # users_movie_count = movies_watched_df.T.notnull().sum().reset_index()
        # users_movie_count.columns = ["userId", "movie_count"]
       
        # movie_ids = users_movie_count[users_movie_count['movie_count'] > 20]['userId'].values.tolist()
        # new_df = movies_watched_df[movies_watched_df.index.isin(movie_ids)]

        # corr_df = new_df.T.corr().unstack().sort_values().drop_duplicates()
        # corr_df = pd.DataFrame(corr_df, columns=["corr"])
        # corr_df.index.names = ['random_user', 'other_users'].reset_index(inplace=True)
            print('working')
        # top_users = corr_df[(corr_df['random_user']==user_id)&(corr_df['corr']>=0.8)]
        # top_users = top_users.loc[:, ['other_users','corr']].rename(columns={"other_users": "userId"})
        # top_users = top_users.reset_index(inplace=True, drop=True)
            print('working')
        # top_users_ratings = top_users.merge(rating, how='inner', on='userId')
        # top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
        # recommendation_df = top_users_ratings.groupby('movieId')[['weighted_rating']].mean().reset_index()
            print('working')
        # final_recommendation_df=recommendation_df[recommendation_df['weighted_rating']>3.5].sort_values(by='weighted_rating', ascending=False)
        # recommendation = final_recommendation_df.merge(movie[["movieId", "title"]])[:5]
        # titles = recommendation['title'].tolist()
            return titles
        except:
            print('No movie to recommend')

    except KeyError:
        print(f'No user with {user_id} id in database')
    except:
        print('Unknown error has occured')
    

def pipeline(movie_path, rating_path,user_id):
    print('Starting...')
    movie_data, rating_data = load_data(movie_path, rating_path)
    print('Load complete..')
    full_df = data_prep(movie_data, rating_data)
    print('Prep complete.')
    recommendation = recommendation_engine(full_df,  user_id, rating_data, movie_data)

    return recommendation
    



data_dir = './Data/archive (1)/'
# gen_scores = pd.read_csv(data_dir + 'genome_scores.csv')
# gen_tags = pd.read_csv(data_dir + 'genome_tags.csv')
# link = pd.read_csv(data_dir + 'link.csv')
# movie = pd.read_csv(data_dir + 'movie.csv')
# rating = pd.read_csv(data_dir + 'rating.csv')
# movie = movie.drop_duplicates()
# rating = rating.drop_duplicates()
# tag = pd.read_csv(data_dir + 'tag.csv')
#tag[['userId','movieId', 'tag']].groupby(['tag']).count().sort_values('userId', ascending = False)

rating_path = data_dir + 'rating.csv'
movie_path = data_dir + 'movie.csv'

#movie, rating = load_data(movie_path, rating_path)
pipeline(movie_path,rating_path, 500 )

Starting...
Load complete..
Prep complete.
Start Recommendation
Pivot complete..
Starting corr
Corr complete..
No movie to recommend


In [3]:
data = data_prep(movie, rating)

In [4]:
data_pivot_table = data.pivot_table(index='userId', columns= 'title', values='rating')

#try:
# random_user = int(pd.Series(data_pivot_table.index).sample(1, random_state=555).values)
user_df = data_pivot_table[data_pivot_table.index == 200]
user_watched_movie = user_df.columns[[user_df.notna().any()]].to_list()

In [47]:
movies_watched_df = data_pivot_table[user_watched_movie]
users_movie_count = movies_watched_df.T.notnull().sum().reset_index()
users_movie_count.columns = ["userId", "movie_count"]
mean_value = np.percentile(users_movie_count['movie_count'],95)
movie_ids = users_movie_count[users_movie_count['movie_count'] > mean_value]['userId'].values.tolist()
new_df = movies_watched_df[movies_watched_df.index.isin(movie_ids)]
corr_df = new_df.T.corr().unstack().sort_values().drop_duplicates()

In [48]:
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['random_user', 'other_users']
corr_df.reset_index(inplace=True)

In [64]:
top_users = corr_df[(corr_df['random_user']==200)&(corr_df['corr']>=0.6)]
top_users = top_users.loc[:, ['other_users','corr']].rename(columns={"other_users": "userId"})
top_users.reset_index(inplace=True, drop=True)
top_users_ratings = top_users.merge(rating, how='inner', on='userId')
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
recommendation_df = top_users_ratings.groupby('movieId')[['weighted_rating']].mean().reset_index()

mean_value = np.percentile(recommendation_df['weighted_rating'],95)
final_recommendation_df=recommendation_df[recommendation_df['weighted_rating']>mean_value].sort_values(by='weighted_rating', ascending=False)
recommendation = final_recommendation_df.merge(movie[["movieId", "title"]])[:5]
titles = recommendation['title'].tolist()
titles

['Syriana (2005)',
 'Lost and Delirious (2001)',
 'Guardians of the Galaxy (2014)',
 'Megamind (2010)',
 'Warriors, The (1979)']

In [41]:
top_users = corr_df[(corr_df['random_user']==200)&(corr_df['corr']>=0.6)]
top_users = top_users.loc[:, ['other_users','corr']].rename(columns={"other_users": "userId"})






AttributeError: 'NoneType' object has no attribute 'merge'

In [3]:
#rating.drop(['timestamp'], axis = 1, inplace = True)
df = rating.merge(movie, on = 'movieId', how = 'inner')

group_df = df.groupby(['movieId']).count()#.sort_values('userId', ascending = False)
idx = group_df[group_df.userId > 1000].index.to_list()
df2 = df[df['movieId'].isin(idx)]

In [4]:
pt = df2.pivot_table(index='userId', columns= 'title', values='rating')

random_user = int(pd.Series(pt.index).sample(1, random_state=555).values)
user_df = pt[pt.index == random_user]
user_watch_movie = user_df.columns[[user_df.notna().any()]].to_list()

In [5]:
movies_watched_df = pt[user_watch_movie]
user_movie_count = movies_watched_df.T.notnull().sum().reset_index()
user_movie_count.columns = ["userId", "movie_count"]

In [6]:
same_movie_id = user_movie_count[user_movie_count['movie_count'] > 20]['userId'].values.tolist()
new_df = movies_watched_df[movies_watched_df.index.isin(same_movie_id)]

In [7]:
corr_df = new_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['random_user', 'other_users']
corr_df.reset_index(inplace=True)

In [8]:
top_users = corr_df[(corr_df['random_user']==random_user)&(corr_df['corr']>=0.8)]
top_users = top_users.loc[:, ['other_users','corr']].rename(columns={"other_users": "userId"})
top_users.reset_index(inplace=True, drop=True)
# top_users

In [9]:
top_users_ratings = top_users.merge(rating, how='inner', on='userId')
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
# top_users_ratings

In [10]:
recommendation_df = top_users_ratings.groupby('movieId')[['weighted_rating']].mean().reset_index()
# recommendation_df

In [11]:
final_recommendation_df=recommendation_df[recommendation_df['weighted_rating']>3.5].sort_values(by='weighted_rating', ascending=False)
final_recommendation_df.reset_index(drop=True,inplace=True)
# final_recommendation_df.shape

In [12]:
final_recommendation_df.merge(movie[["movieId", "title"]])[:5]

,movieId,weighted_rating,title
0,550,4.201677,Threesome (1994)
1,536,4.201677,"Simple Twist of Fate, A (1994)"
2,450,4.201677,With Honors (1994)
3,289,4.183177,Only You (1994)
4,249,4.165989,Immortal Beloved (1994)


In [13]:
recently_5rated_movie_id = rating[(rating['userId']==random_user)&(rating['rating']==5)].sort_values(by='timestamp', ascending=False)['movieId'].values[0]
recently_5rated_movie_id

593

In [14]:
recently_5rated_movie_title= df2[df2['movieId']==recently_5rated_movie_id]['title'].values[0]
recently_5rated_movie_title

'Silence of the Lambs, The (1991)'

In [15]:
user_movie_df_filtered = pt[recently_5rated_movie_title]
user_movie_df_filtered

userId
1         3.5
2         NaN
3         5.0
4         NaN
5         3.0
         ... 
138489    4.0
138490    5.0
138491    NaN
138492    NaN
138493    NaN
Name: Silence of the Lambs, The (1991), Length: 138493, dtype: float64